In [0]:
pip install azure-storage-blob #Package installation.

Python interpreter will be restarted.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-fc8dab3a-3955-45d0-8354-4e6a1786ded4
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
#pip install msal

In [0]:
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
from datetime import datetime, timedelta

# Your Azure Storage details
account_name = "" #Storage account name.
account_key = "" #Storage account key 
container_name = "" #Storage container 
blob_name = ""  # or file name in Data Lake

# Connection string to your Azure Storage Account
connection_string = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"

# Initialize BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get the BlobClient
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Define SAS Token expiration time (e.g., 1 hour from now)
expiry_time = datetime.utcnow() + timedelta(minutes=15)

# Generate the SAS token with read permissions (use write, delete, etc., if needed)
sas_token = generate_blob_sas(
    account_name=account_name,
    container_name=container_name,
    blob_name=blob_name,
    account_key=account_key,
    permission=BlobSasPermissions(read=True),  # Set permissions as needed
    expiry=expiry_time
)

# Construct the full URL to access the blob with SAS token
blob_url_with_sas = f"{blob_client.url}?{sas_token}"

print("Generated SAS Token URL:", blob_url_with_sas)


In [0]:
import msal
import requests

# Azure AD details (Replace with your details)
client_id = ""  # Application (client) ID
client_secret = ""  # Client Secret
tenant_id = ""  # Directory (tenant) ID

# Set the authority URL
authority = f"https://login.microsoftonline.com/{tenant_id}"

# The Azure Data Factory API endpoint
resource = "https://management.azure.com/"

# Scopes for authentication (Azure Data Factory)
scope = ["https://management.azure.com/.default"]  # This is needed for service principal authentication

# Initialize the MSAL Confidential Client
app = msal.ConfidentialClientApplication(
    client_id,
    client_credential=client_secret,
    authority=authority
)

# Request a token from Azure AD
token_response = app.acquire_token_for_client(scopes=scope)

# Check if token is acquired successfully
if "access_token" in token_response:
    access_token = token_response["access_token"]
    print("Access Token:", access_token)
else:
    print("Error: Failed to acquire token")
    print(token_response)


In [0]:
# Set the Data Factory REST API URL
subscription_id = "" #Subscription id
resource_group_name = ""#resource group name 
data_factory_name = "" #asset name 

# API endpoint for getting pipelines
url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{resource_group_name}/providers/Microsoft.DataFactory/factories/{data_factory_name}/pipelines?api-version=2018-06-01"

# Headers including the access token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Make a GET request to Azure Data Factory REST API
response = requests.get(url, headers=headers)

# Print the response (list of pipelines)
if response.status_code == 200:
    print(response.json())  # List of pipelines in Data Factory
else:
    print(f"Error: {response.status_code}, {response.text}")


In [0]:
# import msal
# import requests
# import time

# # Azure AD details (Replace with your details)
# client_id = ""
# client_secret = ""
# tenant_id = ""

# # Authority and scope
# authority = f"https://login.microsoftonline.com/{tenant_id}"
# scope = ["https://management.azure.com/.default"]

# # MSAL client
# app = msal.ConfidentialClientApplication(
#     client_id,
#     client_credential=client_secret,
#     authority=authority
# )

# # Function to get a new token
# def get_access_token():
#     token_response = app.acquire_token_for_client(scopes=scope)
#     if "access_token" in token_response:
#         access_token = token_response["access_token"]
#         expires_in = token_response["expires_in"]  # usually 3600 seconds (1 hour)
#         expires_at = time.time() + expires_in
#         print(f"Access token acquired. Expires in {expires_in // 60} minutes.")
#         return access_token, expires_at
#     else:
#         raise Exception(f"Failed to acquire token: {token_response}")

# # Get initial token
# access_token, expires_at = get_access_token()

# # Refresh buffer (in seconds)
# REFRESH_BUFFER = 5 * 60  # Refresh 5 minutes before actual expiry

# # Example usage loop
# while True:
#     # Check if token is about to expire
#     if time.time() >= (expires_at - REFRESH_BUFFER):
#         print("Refreshing access token...")
#         access_token, expires_at = get_access_token()

#     # --- Your logic using the access token here ---
#     # For demo, just print token's first 10 chars
#     print("Using token:", access_token, "...")
    
#     # Simulate your app doing work every few minutes
#     time.sleep(300)  # wait 5 mins (adjust as needed)
